<span style="font-size:1.5em;"><b>Análisis de bases de datos  
Universidad Nacional de Colombia  
2024-2</b></span>
# <b>Siniestralidad Vial en Bogotá: Análisis de Datos para la Gestión de la Movilidad Segura</b>  

<span style="font-size:0.9em;">Laura Nicole Bermudez Santa - labermudezs@unal.edu.co  
David Sebastián Mendoza Cruz - damendozac@unal.edu.co  
David Alejandro Alquichire Rincón - dalquichire@unal.edu.co  
Laura Nicole Bermudez Santa - labermudezs@unal.edu.co  
Juan David Bernal Vesga - jubernalv@unal.edu.co</span>

----

La seguridad vial es un tema de vital importancia para la planificación urbana y la protección de la vida en las ciudades. En Bogotá, la Secretaría Distrital de Movilidad (SDM) ha recopilado datos detallados sobre siniestralidad vial desde el año 2007, los cuales incluyen información sobre lesionados, accidentes, causas, tipos de vehículos involucrados y actores viales. Estos datos, disponibles en la plataforma Enterprise de la SDM, representan una oportunidad única para analizar y comprender los patrones de accidentalidad en la ciudad, con el fin de contribuir a la toma de decisiones informadas en materia de seguridad vial.

Este proyecto tiene los siguientes objetivos:
1. Analizar y clasificar los accidentes de tránsito registrados en Bogotá en la plataforma de la secretaria de movilidad desde el 2007, enfocándose en la relación entre tipo de vehículo, actores viales involucrados, localidades afectadas, el estado de los incidentes, etc. Con el fin de identificar patrones clave y proporcionar insumos para la toma de decisiones en materia de seguridad vial.  
2. Proporcionar herramientas visuales que guíen la toma de decisiones en materia de seguridad vial en resultado del análisis estadístico realizado.
3. Determinar tendencias y generar pronósticos que permitan anticipar posibles escenarios de siniestralidad en el futuro, mediante una proyección temporal del comportamiento de los accidentes de tránsito, considerando variables como la frecuencia de muertes y accidentes simples.
Buscamos lo que se expone en los objetivos, pero la idea es usar datos abiertos expuestos en la página de la secretaria de movilidad donde tenemos acceso a los siguientes datos de siniestralidad:
Lesionado, accidentes, causa, vehiculo y actor.



----

### **Manipulación y limpieza de las bases de datos individuales**

La Secretaria de movilidad dispone una base de [datos abiertos](https://datos.movilidadbogota.gov.co/maps/ea243e7de8e846c8bd27e47c08771d66/about) amplia en materia de siniestrialidad en Bogotá la cual ha sido alimentada con datos desde el 2007. En esta sección depuraremos los siguientes data sets:
- __*ACCIDENTE*__: nos brinda información detallada de la cantidad de accidentes que se han registrado en la ciudad de bogotá incluyendo sus coordenadas, la clase de choque, fecha y gravedad. (En este dataset se concentra la mayoria de los datos y es de nuestro interes analizarlo a profundidad).
- __*LESIONADO*__: incluye información de las personas involucrada tales como su condición, genero y edad son los datos mas relevantes. (Sin embargo la descripción y el contenido de la información no resulta de nuestro interes).
- __*ACC_VIA*__: caracteristicas relevantes de cada via donde sucedio el accidente, resulta relevante al relacionar el estado de las vías en la ciudad de Bogotá como posible causante de siniestros en ciertos lugares o localidades.
- __*ACC_VEHICULO*__: información general del tipo de vehiculo y el uso de este ya sea publico o particular.
- __*ACC_CAUSA*__: incluye una breve descripción de una posible causa asociada al siniestro.
- __*ACC_ACTOR*__: nos brinda información relevante de la condición del actor causante del accidente.

In [1]:
# Importamos las librerias necesarias en Python

import pandas as pd 
import seaborn as sb 
import numpy as np 
import matplotlib.pyplot as plt 
import requests
%matplotlib inline

<p>Dado que le daremos prioridad al data set que contiene la información general de accidentes <b>ACCIDENTES.CSV</b> comenzaremos limpiando y manipulando la información de <b>ACC_VEHICULO</b>, <b>ACC_CAUSA</b> y <b>ACC_ACTOR</b>.<p>

##### __Traemos *ACC_VEHICULO* que corresponde a la información del vehiculo implicado en el accidente mediante un llamado de API__

In [2]:
# URL de la API
url = "https://sig.simur.gov.co/arcgis/rest/services/Accidentalidad/AccidentalidadAnalisis/FeatureServer/5/query?where=1%3D1&outFields=FORMULARIO,CLASE,SERVICIO&outSR=4326&f=json"

# Hacer la solicitud GET
response = requests.get(url)

# Verificar si la solicitud fue exitosa
if response.status_code == 200:
    # Convertir la respuesta a JSON
    data = response.json()
    print("Datos obtenidos exitosamente")

Datos obtenidos exitosamente


In [3]:
features = data.get('features', [])  # Obtener la lista de features
rows = []  # Lista para almacenar los datos

# Recorrer cada feature y extraer los atributos
for feature in features:
    attributes = feature.get('attributes', {})  # Obtener el diccionario de atributos
    rows.append(attributes)  # Agregar los atributos a la lista

# Convertir la lista de diccionarios a DataFrame
vehiculo_data = pd.DataFrame(rows)
vehiculo_data.head(5)

,FORMULARIO,CLASE,SERVICIO
0,A000035901,AUTOMOVIL,PARTICULAR
1,A000035901,MICROBUS,PUBLICO
2,A000035531,AUTOMOVIL,PARTICULAR
3,A000035139,CAMIONETA,PUBLICO
4,A000035139,AUTOMOVIL,PARTICULAR


In [4]:
# Verificamos si hay valores nulos
vehiculo_data.isna().sum()

FORMULARIO     0
CLASE         27
SERVICIO      85
dtype: int64

In [5]:
# Reemplazamos cadenas vacías o espacios con NaN
vehiculo_data.replace(["", "SIN INFORMACION"," ", "NaN", "null"], np.nan, inplace=True)

# Ahora aplicar dropna()
vehiculo_data.dropna(inplace=True)

# Verficamos que se hayan eliminado correctamente
vehiculo_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1915 entries, 0 to 1999
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   FORMULARIO  1915 non-null   object
 1   CLASE       1915 non-null   object
 2   SERVICIO    1915 non-null   object
dtypes: object(3)
memory usage: 59.8+ KB


El valor en formulario corresponde a un identificador de la información suministrada y a continuación mostramos la variedad de clases de vehiculos que se ven involucrados en accidentes al igual que la clasificación de estos según sus servicios.

In [6]:
print(vehiculo_data['CLASE'].value_counts())
print("")
print(vehiculo_data['SERVICIO'].value_counts())

CLASE
AUTOMOVIL         789
MOTOCICLETA       314
CAMIONETA         248
BUS               208
CAMION, FURGON     98
CAMPERO            83
MICROBUS           77
TRACTOCAMION       38
BUSETA             37
VOLQUETA           23
Name: count, dtype: int64

SERVICIO
PARTICULAR     1022
PUBLICO         847
OFICIAL          45
DIPLOMATICO       1
Name: count, dtype: int64


##### __Traemos *ACC_VIA* que corresponde a la información del estado de la via lugar del accidente mediante un llamado de API__

In [7]:
# URL de la API
url ="https://sig.simur.gov.co/arcgis/rest/services/Accidentalidad/AccidentalidadAnalisis/FeatureServer/6/query?where=1%3D1&outFields=FORMULARIO,GEOMETRICA_A,GEOMETRICA_B,GEOMETRICA_C,UTILIZACION,CALZADAS,CARRILES,MATERIAL,ESTADO,CONDICIONES,ILUMINACION_A,AGENTE_TRANSITO,SEMAFORO&outSR=4326&f=json"

# Hacer la solicitud GET
response = requests.get(url)

# Verificar si la solicitud fue exitosa
if response.status_code == 200:
    # Convertir la respuesta a JSON
    data = response.json()
    print("Datos obtenidos exitosamente")

Datos obtenidos exitosamente


In [8]:
features = data.get('features', [])  # Obtener la lista de features
rows = []  # Lista para almacenar los datos

# Recorrer cada feature y extraer los atributos
for feature in features:
    attributes = feature.get('attributes', {})  # Obtener el diccionario de atributos
    rows.append(attributes)  # Agregar los atributos a la lista

via_data = pd.DataFrame(rows)
via_data.head(5)

,FORMULARIO,GEOMETRICA_A,GEOMETRICA_B,GEOMETRICA_C,UTILIZACION,CALZADAS,CARRILES,MATERIAL,ESTADO,CONDICIONES,ILUMINACION_A,AGENTE_TRANSITO,SEMAFORO
0,712183400,RECTA,PLANO,CON ANDEN,DOBLE SENTIDO,UNA,DOS,ASFALTO,BUENO,SECA,None,None,None
1,712296200,RECTA,None,None,UN SENTIDO,DOS,TRES O MAS,ASFALTO,BUENO,SECA,None,None,OPERANDO
2,712323300,RECTA,PLANO,CON ANDEN,UN SENTIDO,DOS,DOS,ASFALTO,BUENO,SECA,CON,None,None
3,726927900,RECTA,PLANO,CON ANDEN,DOBLE SENTIDO,UNA,DOS,ASFALTO,BUENO,SECA,CON,None,OPERANDO
4,712905400,RECTA,PLANO,CON ANDEN,DOBLE SENTIDO,UNA,DOS,ASFALTO,BUENO,HUMEDA,CON,None,None


In [9]:
# Revisamos valores nulos 
via_data.isna().sum()

FORMULARIO            0
GEOMETRICA_A         53
GEOMETRICA_B        200
GEOMETRICA_C        328
UTILIZACION          98
CALZADAS            117
CARRILES             94
MATERIAL            104
ESTADO              138
CONDICIONES          89
ILUMINACION_A       552
AGENTE_TRANSITO     729
SEMAFORO           1128
dtype: int64

In [10]:
# Reemplazar cadenas vacías o espacios con NaN
via_data.replace(["", "SIN INFORMACION"," ", "NaN", "null"], np.nan, inplace=True)

# Ahora aplicar dropna()
via_data.dropna(inplace=True)

# Verficamos que se hayan eliminado correctamente
via_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 204 entries, 36 to 1969
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   FORMULARIO       204 non-null    object
 1   GEOMETRICA_A     204 non-null    object
 2   GEOMETRICA_B     204 non-null    object
 3   GEOMETRICA_C     204 non-null    object
 4   UTILIZACION      204 non-null    object
 5   CALZADAS         204 non-null    object
 6   CARRILES         204 non-null    object
 7   MATERIAL         204 non-null    object
 8   ESTADO           204 non-null    object
 9   CONDICIONES      204 non-null    object
 10  ILUMINACION_A    204 non-null    object
 11  AGENTE_TRANSITO  204 non-null    object
 12  SEMAFORO         204 non-null    object
dtypes: object(13)
memory usage: 22.3+ KB


El valor en formulario corresponde a un identificador de la información suministrada y a continuación mostramos los posibles valores de los siguientes atributos semaforo -se indica si el semaforo se encontraba operando-, Estado -estado general de la via- Geometrica_A, Geometrica_B, Geometrica_C -descripciones de la via-, utilización -sentidos de funcionamientos-, calzadas, carriles, material de la via, condiciones al momento del accidente, iluminación y la presencia de un agente de transito.

In [11]:
print(via_data["SEMAFORO"].value_counts())
print("")
print(via_data["ESTADO"].value_counts())
print("")
print(via_data["GEOMETRICA_A"].value_counts())
print(via_data["GEOMETRICA_B"].value_counts())
print(via_data["GEOMETRICA_C"].value_counts())
print("")
print(via_data["UTILIZACION"].value_counts())
print("")
print(via_data["CALZADAS"].value_counts())
print("")
print(via_data["CARRILES"].value_counts())
print("")
print(via_data["MATERIAL"].value_counts())
print("")
print(via_data["CONDICIONES"].value_counts())
print("")
print(via_data["ILUMINACION_A"].value_counts())
print("")
print(via_data["AGENTE_TRANSITO"].value_counts())

SEMAFORO
OPERANDO    204
Name: count, dtype: int64

ESTADO
BUENO           197
CON HUECOS        3
HUNDIMIENTOS      2
PARCHADA          1
RIZADA            1
Name: count, dtype: int64

GEOMETRICA_A
RECTA    199
CURVA      5
Name: count, dtype: int64
GEOMETRICA_B
PLANO        198
PENDIENTE      6
Name: count, dtype: int64
GEOMETRICA_C
CON ANDEN     182
CON BERMAS     22
Name: count, dtype: int64

UTILIZACION
UN SENTIDO       127
DOBLE SENTIDO     76
REVERSIBLE         1
Name: count, dtype: int64

CALZADAS
UNA             126
DOS              57
CUATRO O MAS     14
TRES O MAS        5
VARIABLE          2
Name: count, dtype: int64

CARRILES
DOS             127
TRES O MAS       47
UNO              13
CUATRO O MAS     12
VARIABLE          5
Name: count, dtype: int64

MATERIAL
ASFALTO     188
CONCRETO     14
AFIRMADO      2
Name: count, dtype: int64

CONDICIONES
SECA      188
HUMEDA     16
Name: count, dtype: int64

ILUMINACION_A
CON    174
SIN     30
Name: count, dtype: int64

AGENTE_TRANS

Como se puede observar podriamos convertir la información del atributo *AGENTE_Transito* y *ILUMINACION_A* en un tipo de dato númerico binario haciendo uso de dummies.

In [12]:
# Usamos dummies para la información de la columna ILUMINACION_A
# 1 indica que si habia iluminación en la via y el 0 indica la ausencia de iluminación.
dummies_iluminacion = pd.get_dummies(via_data["ILUMINACION_A"],dtype=int)
via_data = via_data.join(dummies_iluminacion)
via_data.drop("ILUMINACION_A", axis=1, inplace=True)

via_data.drop("SIN", axis=1, inplace=True)
via_data.rename(columns={"CON": "ILUMINACION"}, inplace=True)


# Usamos dummies para la información de la columna AGENTES_TRANSITO
# 1 indica que si habian agentes de transito en la via y el 0 indica la ausencia de estos.
dummies_agentes = pd.get_dummies(via_data["AGENTE_TRANSITO"],dtype=int)
via_data = via_data.join(dummies_agentes)
via_data.drop("AGENTE_TRANSITO", axis=1, inplace=True)

via_data.drop("NO", axis=1, inplace=True)
via_data.rename(columns={"SI": "AGENTE_TRANSITO"}, inplace=True)

via_data.head(5)

,FORMULARIO,GEOMETRICA_A,GEOMETRICA_B,GEOMETRICA_C,UTILIZACION,CALZADAS,CARRILES,MATERIAL,ESTADO,CONDICIONES,SEMAFORO,ILUMINACION,AGENTE_TRANSITO
36,833828800,RECTA,PLANO,CON ANDEN,DOBLE SENTIDO,UNA,DOS,ASFALTO,BUENO,SECA,OPERANDO,1,0
40,828670000,RECTA,PLANO,CON ANDEN,UN SENTIDO,UNA,DOS,ASFALTO,BUENO,SECA,OPERANDO,1,0
42,834011400,RECTA,PLANO,CON ANDEN,UN SENTIDO,UNA,DOS,ASFALTO,BUENO,SECA,OPERANDO,1,0
55,833888800,RECTA,PLANO,CON ANDEN,UN SENTIDO,UNA,DOS,ASFALTO,BUENO,SECA,OPERANDO,1,0
56,828668700,RECTA,PLANO,CON ANDEN,DOBLE SENTIDO,CUATRO O MAS,VARIABLE,ASFALTO,BUENO,SECA,OPERANDO,1,0


##### __Traemos *ACC_CAUSA* que corresponde a la información de la causa del accidente estipulada en el formulario mediante un llamado de API__

In [13]:
# URL de la API
url = "https://sig.simur.gov.co/arcgis/rest/services/Accidentalidad/AccidentalidadAnalisis/FeatureServer/4/query?where=1%3D1&outFields=FORMULARIO,NOMBRE,TIPO_CAUSA,TIPO&outSR=4326&f=json"

# Hacer la solicitud GET
response = requests.get(url)

# Verificar si la solicitud fue exitosa
if response.status_code == 200:
    # Convertir la respuesta a JSON
    data = response.json()
    print("Datos obtenidos exitosamente")

Datos obtenidos exitosamente


In [14]:
features = data.get('features', [])  # Obtener la lista de features
rows = []  # Lista para almacenar los datos

# Recorrer cada feature y extraer los atributos
for feature in features:
    attributes = feature.get('attributes', {})  # Obtener el diccionario de atributos
    rows.append(attributes)  # Agregar los atributos a la lista

causa_data = pd.DataFrame(rows)
causa_data.head(5)

,FORMULARIO,NOMBRE,TIPO_CAUSA,TIPO
0,A000033002,TRANSITAR DISTANTE DE LA ACERA U ORILLA DE LA ...,CONDUCTOR,CE
1,A000032697,DESOBEDECER SEÑALES,CONDUCTOR,CG
2,A000034603,OTRA,CONDUCTOR,CG
3,A000035837,NO MANTENER DISTANCIA DE SEGURIDAD,CONDUCTOR,CG
4,A000034539,NO MANTENER DISTANCIA DE SEGURIDAD,CONDUCTOR,CG


In [15]:
causa_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   FORMULARIO  2000 non-null   object
 1   NOMBRE      2000 non-null   object
 2   TIPO_CAUSA  2000 non-null   object
 3   TIPO        2000 non-null   object
dtypes: object(4)
memory usage: 62.6+ KB


El valor en formulario corresponde a un identificador de la información suministrada y a continuación mostramos los posibles valores de los siguientes atributos tipos de causa y tipo (de la cual no obtenemos alguna información con respecto a las siglas que representan).

In [16]:
print(causa_data["TIPO_CAUSA"].value_counts())
print("")
print(causa_data["TIPO"].value_counts())

TIPO_CAUSA
CONDUCTOR    1748
PEATON        202
VIA            28
PASAJERO       12
VEHICULO       10
Name: count, dtype: int64

TIPO
CG    1718
PE     202
CE      30
VI      28
PA      12
VH      10
Name: count, dtype: int64


##### __Traemos *ACC_ACTOR_VIAL* que corresponde a la información general del actor del accidente mediante un llamado de API__

In [17]:
# URL de la API
url = "https://sig.simur.gov.co/arcgis/rest/services/Accidentalidad/AccidentalidadAnalisis/FeatureServer/3/query?where=1%3D1&outFields=FORMULARIO,CONDICION,ESTADO,MUERTE_POSTERIOR,GENERO,EDAD&outSR=4326&f=json"

# Hacer la solicitud GET
response = requests.get(url)

# Verificar si la solicitud fue exitosa
if response.status_code == 200:
    # Convertir la respuesta a JSON
    data = response.json()
    print("Datos obtenidos exitosamente")

Datos obtenidos exitosamente


In [18]:
features = data.get('features', [])  # Obtener la lista de features
rows = []  # Lista para almacenar los datos

# Recorrer cada feature y extraer los atributos
for feature in features:
    attributes = feature.get('attributes', {})  # Obtener el diccionario de atributos
    rows.append(attributes)  # Agregar los atributos a la lista

actor_data = pd.DataFrame(rows)
actor_data.head(5)

,FORMULARIO,CONDICION,ESTADO,MUERTE_POSTERIOR,GENERO,EDAD
0,728478600,CONDUCTOR,ILESO,None,SIN INFORMACION,NaN
1,728479300,CONDUCTOR,ILESO,None,SIN INFORMACION,NaN
2,728479400,CONDUCTOR,HERIDO,N,FEMENINO,41.0
3,728479700,CONDUCTOR,ILESO,None,MASCULINO,39.0
4,728480500,MOTOCICLISTA,HERIDO,N,MASCULINO,36.0


In [19]:
# Revisamos valores nulos 
actor_data.isna().sum()

FORMULARIO             0
CONDICION              0
ESTADO                 2
MUERTE_POSTERIOR    1392
GENERO                 0
EDAD                 698
dtype: int64

In [20]:
# Reemplazar cadenas vacías o espacios con NaN
actor_data.replace(["", "SIN INFORMACION"," ", "NaN", "null"], np.nan, inplace=True)

# Ahora aplicar dropna()
actor_data.dropna(inplace=True)

In [21]:
actor_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 587 entries, 2 to 1997
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   FORMULARIO        587 non-null    object 
 1   CONDICION         587 non-null    object 
 2   ESTADO            587 non-null    object 
 3   MUERTE_POSTERIOR  587 non-null    object 
 4   GENERO            587 non-null    object 
 5   EDAD              587 non-null    float64
dtypes: float64(1), object(5)
memory usage: 32.1+ KB


El valor en formulario corresponde a un identificador de la información suministrada y a continuación mostramos los posibles valores de los siguientes atributos estado del actor, genero y si el actor murio post accidente.

In [22]:
print(actor_data["ESTADO"].value_counts())
print("")
print(actor_data["GENERO"].value_counts())
print("")
print(actor_data["MUERTE_POSTERIOR"].value_counts()   )

ESTADO
HERIDO    587
Name: count, dtype: int64

GENERO
MASCULINO    541
FEMENINO      46
Name: count, dtype: int64

MUERTE_POSTERIOR
N    587
Name: count, dtype: int64


Como se puede observar podriamos convertir la información del atributo *GENERO* y *MUERTE_POSTERIOR* en un tipo de dato númerico binario haciendo uso de dummies.

In [23]:
# Usamos dummies para la información de la columna GENERO
# 1 indica que el genero es Femenino y el 0 indica Maculino.
dummies_genero = pd.get_dummies(actor_data["GENERO"],dtype=int)
actor_data = actor_data.join(dummies_genero)
actor_data.drop("GENERO", axis=1, inplace=True)

actor_data.drop("MASCULINO", axis=1, inplace=True)
actor_data.rename(columns={"FEMENINO": "GENERO"}, inplace=True)


# Usamos dummies para la información de la columna MUERTE_POSTERIOR
# 1 indica que el actor murio posterior al accidente
# y el 0 indica que el actor vial estuvo vivo durante el accidente.
dummies_muertePosterior = pd.get_dummies(actor_data["MUERTE_POSTERIOR"],dtype=int)
actor_data = actor_data.join(dummies_muertePosterior)
actor_data.drop("MUERTE_POSTERIOR", axis=1, inplace=True)

actor_data.drop("N", axis=1, inplace=True)
actor_data.rename(columns={"S": "MUERTE_POSTERIOR"}, inplace=True)

actor_data.head(5)

,FORMULARIO,CONDICION,ESTADO,EDAD,GENERO
2,728479400,CONDUCTOR,HERIDO,41.0,1
4,728480500,MOTOCICLISTA,HERIDO,36.0,0
7,728485000,CONDUCTOR,HERIDO,29.0,0
11,728492800,CONDUCTOR,HERIDO,44.0,0
19,728501000,CICLISTA,HERIDO,28.0,0


##### __Traemos *ACCIDENTES* que corresponde a la información general del siniestro vial mediante un llamado de API__

Estos data sets nos dan información detallada sobre algunos aspectos involucrados en el accidente. Sin embargo la mayor concentración de información se encuentra en __*ACCIDENTES*__, la cual resulta de nuestro mayor interes en este caso.

In [24]:
# URL de la API
url = "https://sig.simur.gov.co/arcgis/rest/services/Accidentalidad/AccidentalidadAnalisis/FeatureServer/2/query?where=1%3D1&outFields=FORMULARIO,FECHA_OCURRENCIA_ACC,HORA_OCURRENCIA_ACC,ANO_OCURRENCIA_ACC,MES_OCURRENCIA_ACC,DIA_OCURRENCIA_ACC,DIRECCION,GRAVEDAD,CLASE_ACC,LOCALIDAD,MUNICIPIO,LATITUD,LONGITUD&outSR=4326&f=json" 

# Hacer la solicitud GET
response = requests.get(url)

# Verificar si la solicitud fue exitosa
if response.status_code == 200:
    # Convertir la respuesta a JSON
    data = response.json()
    print("Datos obtenidos exitosamente")

Datos obtenidos exitosamente


In [25]:
features = data.get('features', [])  # Obtener la lista de features
rows = []  # Lista para almacenar los datos

# Recorrer cada feature y extraer los atributos
for feature in features:
    attributes = feature.get('attributes', {})  # Obtener el diccionario de atributos
    rows.append(attributes)  # Agregar los atributos a la lista

accidentes_data = pd.DataFrame(rows)
accidentes_data.head(5)

,FORMULARIO,FECHA_OCURRENCIA_ACC,HORA_OCURRENCIA_ACC,ANO_OCURRENCIA_ACC,MES_OCURRENCIA_ACC,DIA_OCURRENCIA_ACC,DIRECCION,GRAVEDAD,CLASE_ACC,LOCALIDAD,MUNICIPIO,LATITUD,LONGITUD
0,A001390689,1644814800000,19:30:00,2022.0,FEBRERO,LUNES,DG 73 G - KR 79 A S 02,CON HERIDOS,ATROPELLO,BOSA,BOGOTA DC,4.607566,-74.201473
1,A00410350,1220850000000,08:20:00,2008.0,SEPTIEMBRE,LUNES,TR 52-1B 02,CON MUERTOS,CHOQUE,PUENTE ARANDA,BOGOTA DC,4.610252,-74.118635
2,A000757293,1516856400000,21:20:00,2018.0,ENERO,JUEVES,CL 52-KR 13A S 2,CON HERIDOS,CHOQUE,TUNJUELITO,BOGOTA DC,4.563616,-74.126296
3,727366900,1188277200000,07:30:00,2007.0,AGOSTO,MARTES,AC 116-70H 0,SOLO DANOS,CHOQUE,SUBA,BOGOTA DC,4.699019,-74.076723
4,A1061345,1337058000000,05:50:00,2012.0,MAYO,MARTES,CL 26-KR 79A S 02,SOLO DANOS,CHOQUE,KENNEDY,BOGOTA DC,4.629726,-74.152551


In [26]:
# Revisamos valores nulos
accidentes_data.isna().sum()

FORMULARIO              0
FECHA_OCURRENCIA_ACC    0
HORA_OCURRENCIA_ACC     0
ANO_OCURRENCIA_ACC      0
MES_OCURRENCIA_ACC      0
DIA_OCURRENCIA_ACC      0
DIRECCION               0
GRAVEDAD                0
CLASE_ACC               0
LOCALIDAD               0
MUNICIPIO               0
LATITUD                 0
LONGITUD                0
dtype: int64

In [27]:
# Reemplazar cadenas vacías o espacios con NaN
accidentes_data.replace(["", " ", "NaN", "null"], np.nan, inplace=True)

# Ahora aplicar dropna()
accidentes_data.dropna(inplace=True)

In [28]:
accidentes_data.dropna()
accidentes_data.info()
# Obtenemos finalmente una tabla sin valores nulos

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   FORMULARIO            2000 non-null   object 
 1   FECHA_OCURRENCIA_ACC  2000 non-null   int64  
 2   HORA_OCURRENCIA_ACC   2000 non-null   object 
 3   ANO_OCURRENCIA_ACC    2000 non-null   float64
 4   MES_OCURRENCIA_ACC    2000 non-null   object 
 5   DIA_OCURRENCIA_ACC    2000 non-null   object 
 6   DIRECCION             2000 non-null   object 
 7   GRAVEDAD              2000 non-null   object 
 8   CLASE_ACC             2000 non-null   object 
 9   LOCALIDAD             2000 non-null   object 
 10  MUNICIPIO             2000 non-null   object 
 11  LATITUD               2000 non-null   float64
 12  LONGITUD              2000 non-null   float64
dtypes: float64(3), int64(1), object(9)
memory usage: 203.3+ KB


In [29]:
# Información que no es relevante para el análisis, todos los registros son en bogotá
accidentes_data.drop(columns=['MUNICIPIO'],inplace=True) 
accidentes_data.head(5)

,FORMULARIO,FECHA_OCURRENCIA_ACC,HORA_OCURRENCIA_ACC,ANO_OCURRENCIA_ACC,MES_OCURRENCIA_ACC,DIA_OCURRENCIA_ACC,DIRECCION,GRAVEDAD,CLASE_ACC,LOCALIDAD,LATITUD,LONGITUD
0,A001390689,1644814800000,19:30:00,2022.0,FEBRERO,LUNES,DG 73 G - KR 79 A S 02,CON HERIDOS,ATROPELLO,BOSA,4.607566,-74.201473
1,A00410350,1220850000000,08:20:00,2008.0,SEPTIEMBRE,LUNES,TR 52-1B 02,CON MUERTOS,CHOQUE,PUENTE ARANDA,4.610252,-74.118635
2,A000757293,1516856400000,21:20:00,2018.0,ENERO,JUEVES,CL 52-KR 13A S 2,CON HERIDOS,CHOQUE,TUNJUELITO,4.563616,-74.126296
3,727366900,1188277200000,07:30:00,2007.0,AGOSTO,MARTES,AC 116-70H 0,SOLO DANOS,CHOQUE,SUBA,4.699019,-74.076723
4,A1061345,1337058000000,05:50:00,2012.0,MAYO,MARTES,CL 26-KR 79A S 02,SOLO DANOS,CHOQUE,KENNEDY,4.629726,-74.152551


Revisamos el tipo de información de algunos atributos relevantes como la clase de accidentes que corresponde a la clasificación del tipo de accidente y la gravedad del accidente.

In [30]:
print(accidentes_data['CLASE_ACC'].value_counts())
print("")
print(accidentes_data['GRAVEDAD'].value_counts())

CLASE_ACC
CHOQUE               1577
ATROPELLO             300
CAIDA DE OCUPANTE      70
OTRO                   27
VOLCAMIENTO            22
AUTOLESION              4
Name: count, dtype: int64

GRAVEDAD
CON HERIDOS    1081
SOLO DANOS      872
CON MUERTOS      47
Name: count, dtype: int64


----
### **Análisis estadistico basico del dataset **